In [1]:
import anndata as ad
import numpy as np
import scanpy as sc
import biolord

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata_train_path= "/lustre/groups/ml01/workspace/ot_perturbation/data/norman/adata_train_1_seen_genes.h5ad"
adata_test_path= "/lustre/groups/ml01/workspace/ot_perturbation/data/norman/adata_test_1_seen_genes.h5ad"
adata_ood_path= "/lustre/groups/ml01/workspace/ot_perturbation/data/norman/adata_ood_1_seen_genes.h5ad"

In [3]:
ad_train = ad.read_h5ad(adata_train_path)
ad_test = ad.read_h5ad(adata_test_path)
ad_ood = ad.read_h5ad(adata_ood_path)

In [4]:
ad_train.uns.keys()

dict_keys(['doi', 'hvg', 'is_ood_colors', 'leiden', 'log1p', 'neighbors', 'pca', 'pert2neighbor', 'pert2neighbor_prepared', 'preprocessing_nb_link', 'rank_genes_groups_cov_all', 'umap'])

In [5]:
pert2neighbor = np.asarray([val for val in ad_train.uns["pert2neighbor"].values()])

In [6]:
gene_emb_pca = sc.pp.pca(pert2neighbor)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [7]:
gene_emb_dict = {k: gene_emb_pca[i, :] for i,k in enumerate(ad_train.uns["pert2neighbor"].keys())}

In [8]:
adata = ad.concat((ad_train, ad_test, ad_ood), label="split", keys=["train", "test", "ood"])

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [9]:
import numpy as np

def get_gene_emb(x):
    return np.concatenate((gene_emb_dict[x["gene_1"]][:, None], gene_emb_dict[x["gene_2"]][:, None]), axis=0).T

gene_emb_list = adata.obs.apply(get_gene_emb, axis=1).tolist()
gene_emb_array = np.array(gene_emb_list).squeeze()
adata.obsm["gene_emb"] = gene_emb_array


In [10]:
ordered_attributes_key="gene_emb"

In [11]:
module_params = {
    "decoder_width": 32,
    "decoder_depth": 2,
    "decoder_activation": True,
    "attribute_nn_width": 64,
    "attribute_nn_depth": 2,
    "attribute_nn_activation": True,
    "use_batch_norm": False,
    "use_layer_norm": False,
    "unknown_attribute_noise_param": 0.2,
    "seed": 42,
    "n_latent_attribute_ordered": 32,
    "n_latent_attribute_categorical": 16,
    "reconstruction_penalty": 10000.0,
    "unknown_attribute_penalty": 10000.0,
    "attribute_dropout_rate": 0.0,
    "unknown_attributes": False
}

trainer_params = {
    "n_epochs_warmup": 0,
    "latent_lr": 0.1,
    "latent_wd": 0.00001,
    "decoder_lr": 0.01,
    "decoder_wd": 0.01,
    "attribute_nn_lr": 0.001,
    "attribute_nn_wd": 4e-8,
    "step_size_lr": 45,
    "cosine_scheduler": True,
    "scheduler_final_lr": 1e-5,
}

In [12]:
varying_arg = {
    "seed": 42,
    "unknown_attribute_noise_param": 0.2, 
    "use_batch_norm": False,
    "use_layer_norm": False, 
    "step_size_lr": 45, 
    "attribute_dropout_rate": 0.0, 
    "cosine_scheduler":True,
    "scheduler_final_lr":1e-5,
    "n_latent":32, 
    "n_latent_attribute_ordered": 32,
    "reconstruction_penalty": 10000.0,
    "attribute_nn_width": 64,
    "attribute_nn_depth" :2, 
    "attribute_nn_lr": 0.001, 
    "attribute_nn_wd": 4e-8,
    "latent_lr": 0.01,
    "latent_wd": 0.00001,
    "decoder_width": 32,
    "decoder_depth": 2,  
    "decoder_activation": True,
    "attribute_nn_activation": True,
    "unknown_attributes": False,
    "decoder_lr": 0.01,
    "decoder_wd": 0.01,
    "max_epochs":200,
    "early_stopping_patience": 200,
    "ordered_attributes_key": "perturbation_neighbors1",
    "n_latent_attribute_categorical": 16,
}

In [13]:
biolord.Biolord.setup_anndata(
    adata,
    ordered_attributes_keys=[ordered_attributes_key],
    categorical_attributes_keys=None,
    retrieval_attribute_key=None,
)

INFO     Generating sequential column names                                                                        


In [14]:
model = biolord.Biolord(
        adata=adata,
        n_latent=varying_arg["n_latent"],
        model_name="norman",
        module_params=module_params,
        train_classifiers=False,
        split_key="split"
    )

[rank: 0] Seed set to 42


In [15]:
model.train(
    max_epochs=int(varying_arg["max_epochs"]),
    batch_size=32,
    plan_kwargs=trainer_params,
    early_stopping=True,
    early_stopping_patience=int(varying_arg["early_stopping_patience"]),
    check_val_every_n_epoch=5,
    num_workers=1,
    enable_checkpointing=False
)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_biol ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_biol ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/

Epoch 2/200:   0%|          | 1/200 [00:32<1:46:15, 32.04s/it, v_num=1, val_generative_mean_accuracy=0.99, val_generative_var_accuracy=-39, val_biolord_metric=-19, val_LOSS_KEYS.RECONSTRUCTION=697, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=0]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 3/200:   1%|          | 2/200 [00:51<1:21:39, 24.75s/it, v_num=1, val_generative_mean_accuracy=0.991, val_generative_var_accuracy=-40.5, val_biolord_metric=-19.8, val_LOSS_KEYS.RECONSTRUCTION=691, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=0, generative_mean_accuracy=0, generative_var_accuracy=0, biolord_metric=0, reconstruction_loss=718, unknown_attribute_penalty_loss=0]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 200/200: 100%|██████████| 200/200 [1:03:28<00:00, 18.68s/it, v_num=1, val_generative_mean_accuracy=0.992, val_generative_var_accuracy=-42.3, val_biolord_metric=-20.7, val_LOSS_KEYS.RECONSTRUCTION=684, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=0, generative_mean_accuracy=0, generative_var_accuracy=0, biolord_metric=0, reconstruction_loss=682, unknown_attribute_penalty_loss=0]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [1:03:28<00:00, 19.04s/it, v_num=1, val_generative_mean_accuracy=0.992, val_generative_var_accuracy=-42.3, val_biolord_metric=-20.7, val_LOSS_KEYS.RECONSTRUCTION=684, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=0, generative_mean_accuracy=0, generative_var_accuracy=0, biolord_metric=0, reconstruction_loss=682, unknown_attribute_penalty_loss=0]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [16]:
model.save("/lustre/groups/ml01/workspace/ot_perturbation/models/biolord/norman/training_1", overwrite=True)

In [17]:
import torch

def bool2idx(x):
    """
    Returns the indices of the True-valued entries in a boolean array `x`
    """
    return np.where(x)[0]

def repeat_n(x, n):
    """
    Returns an n-times repeated version of the Tensor x,
    repetition dimension is axis 0
    """
    # copy tensor to device BEFORE replicating it n times
    device = "cuda" if torch.cuda.is_available() else "cpu"
    return x.to(device).view(1, -1).repeat(n, 1)

In [18]:
idx_test_control = np.where(
    (adata.obs["split"] == "test") & (adata.obs["control"] == 1)
)[0]

adata_test_control = adata[idx_test_control].copy()

idx_ood = np.where((adata.obs["split"] == "ood"))[0]

adata_ood = adata[idx_ood].copy()

In [19]:
dataset_ood = model.get_dataset(adata_ood)

INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


In [20]:
import pandas as pd
pert_categories_index = pd.Index(adata.obs["condition"].values, dtype="category")

In [21]:
import pandas as pd
from tqdm import tqdm

def compute_prediction(
    model,
    adata,
    dataset,
    adata_test_control,
    n_obs=500
):
    pert_categories_index = pd.Index(adata.obs["condition"].values, dtype="category")


    layer = "X" if "X" in dataset else "layers"
    predictions_dict = {}
    for perturbation in tqdm(np.unique(pert_categories_index.values)
    ):
        dataset_control = model.get_dataset(adata_test_control)

        bool_category = pert_categories_index.get_loc(perturbation)
        idx_all = bool2idx(bool_category)
        idx = idx_all[0]
                    
        dataset_comb = {}

        dataset_comb[layer] = dataset_control[layer].to(model.device)
        dataset_comb["ind_x"] = dataset_control["ind_x"].to(model.device)
        for key in dataset_control:
            if key not in [layer, "ind_x"]:
                dataset_comb[key] = repeat_n(dataset[key][idx, :], n_obs)

            
        pred_mean, pred_std = model.module.get_expression(dataset_comb)
        samples = torch.normal(pred_mean, pred_std)

        predictions_dict[perturbation] = samples.detach().cpu().numpy()
    return predictions_dict

In [22]:
biolord_prediction = compute_prediction(
    model=model,
    adata=adata_ood,
    dataset=dataset_ood,
    adata_test_control=adata_test_control)

  0%|          | 0/24 [00:00<?, ?it/s]

INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


100%|██████████| 24/24 [00:01<00:00, 15.74it/s]


In [23]:
all_data = []
conditions = []

for condition, array in biolord_prediction.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_pred = ad.AnnData(X=all_data_array, obs=obs_data)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [24]:
adata_pred.write_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/models/biolord/norman/biolord_output_ood_1_seen.h5ad")

In [25]:
idx_test = np.where((adata.obs["split"] == "test"))[0]

adata_test = adata[idx_test].copy()

dataset_test = model.get_dataset(adata_test)
biolord_prediction = compute_prediction(
    model=model,
    adata=adata_test,
    dataset=dataset_test,
    adata_test_control=adata_test_control)
all_data = []
conditions = []

for condition, array in biolord_prediction.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_pred = ad.AnnData(X=all_data_array, obs=obs_data)
adata_pred.write_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/models/biolord/norman/biolord_output_test_1_seen.h5ad")

INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


100%|██████████| 125/125 [00:01<00:00, 118.34it/s]
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
